# Low z LLS (Lehner+13, Wotta+16)

In [20]:
# imports
from scipy.io import readsav

import pyigm
from pyigm.abssys.lls import LLSSystem
from pyigm.surveys.llssurvey import LLSSurvey
pyigm_path = pyigm.__path__[0]
from pyigm.surveys import utils as pyiu


## Lehner+13

### Metals

From
http://iopscience.iop.org/0004-637X/770/2/138/suppdata/apj472363t2_mrt.txt

### Metallicity

In [2]:
l13w16 = readsav(pyigm_path+'/data/LLS/Literature/wotta16_final.save')

### RA/DEC

In [3]:
l13_radec = {'J0943+0531': 'J094331.61+053131.4', 
             'J1419+4207': 'J141910.20+420746.9', 
             'J1435+3604': 'J143511.53+360437.2', 
             'PG1116+215': 'J111908.67+211918.0',
             'PG1522+101': 'J152424.52+095829.1', 
             'PG1338+416': 'J134100.78+412314.0', 
             'HE0439-5254': 'J044012.04-524817.65',        
             'PG1407+265': 'J140923.91+261821.1', 
             'PG1216+069': 'J121920.92+063838.5', 
             'PKS0405-123': 'J040748.44-121136.7',
             'J1619+3342': 'J161916.54+334238.4', 
             'PKS0637-752': 'J063546.51-751616.7', 
             'SBS1122+594': 'J112553.79+591021.5',
             'HE0153-4520': 'J015513.22-450611.6', 
             'PHL1377': 'J023507.34-040205.3', 
             'PKS0552-640': 'J055224.51-640210.8',
             'PG1630+377': 'J163201.11+373750.0', 
             'PKS1302-102': 'J130533.01-103319.4', 
             'PG1206+459': 'J120858.00+454035.4', 
             'PG1634+706': 'J163428.99+703132.4'}

### $z_{\rm em}$

In [4]:
l13_zem =  {'J0943+0531': 0.56449,
             'J1419+4207': 0.87350,
             'J1435+3604': 0.42860,
             'PG1116+215': 0.17650,
             'PG1522+101': 1.32825,
             'PG1338+416': 1.21420,
             'HE0439-5254': 1.05300,
             'PG1407+265': 0.94,
             'PG1216+069': 0.3313,
             'PKS0405-123': 0.5726,
             'J1619+3342': 0.47093,
             'PKS0637-752': 0.651,
             'SBS1122+594': 0.85242, 
             'HE0153-4520': 0.451,
             'PHL1377': 1.4398,
             'PKS0552-640': 0.680,
             'PG1630+377': 1.47465,
             'PKS1302-102': 0.2784,
             'PG1206+459': 1.16200,
             'PG1634+706': 1.337 }

In [5]:
l13w16['znl']

array([ 0.35420001,  0.42559999,  0.38780001,  0.13850001,  0.51849997,
        0.3488    ,  0.6153    ,  0.53460002,  0.68279999,  0.28889999,
        0.2823    ,  0.1672    ,  0.68650001,  0.2694    ,  0.46849999,
        0.55739999,  0.2261    ,  0.373     ,  0.72920001,  0.7392    ,
        0.34509999,  0.27399999,  0.0985    ,  0.92699999,  1.03999996], dtype=float32)

### Build the systems

In [6]:
L13_lls = []
for ii,qso in enumerate(l13w16['sightlinenl']):
    # init
    lls = LLSSystem(radec=l13_radec[qso],zem=l13_zem[qso],
                    zabs=l13w16['znl'][ii],NHI=l13w16['nhinl'][ii],
                    sig_NHI=l13w16['enhinl'][ii], 
                    vlim=[-500,500]*u.km/u.s)
    # Metallicity
    lls.ZH = l13w16['metnl'][ii]
    lls.sig_ZH = (l13w16['em_metnl'][ii], l13w16['ep_metnl'][ii])
    # Append
    L13_lls.append(lls)

### Finish

In [7]:
L13 = LLSSurvey()
L13._abs_sys = L13_lls

In [8]:
L13.ZH

array([-1.29999995, -1.39999998, -1.39999998, -0.5       , -0.40000001,
       -0.75      , -0.30000001, -0.2       , -1.79999995, -0.64999998,
       -1.64999998,  0.1       ,  0.1       , -1.60000002, -0.40000001,
       -1.04999995, -0.80000001, -1.85000002, -2.        , -1.45000005,
       -1.60000002, -1.71000004, -1.60000002,  0.30000001, -1.39999998], dtype=float32)

## Wotta+16

### Big Table

In [9]:
w16_mtl = Table.read(pyigm_path+'/data/LLS/Literature/wotta16_metallicities.ascii',format='ascii', guess=False)

In [10]:
w16_mtl[0:5]

Target,z_em,RA,DEC,z_abs,logNHI,elogNHI,logNMgII,ep_logNMgII,em_logNMgII,[X/H],ep_[X/H],em_[X/H],68%[X/H]_low,68%[X/H]_high,95%[X/H]_low,95%[X/H]_high
str11,float64,str12,str11,float64,float64,float64,str5,float64,float64,float64,float64,float64,str7,str7,str7,str7
J0800+4212,0.99407,+08:00:03.90,+42:12:53.2,0.598,16.95,0.2,11.75,0.17,0.3,-1.99,0.3,0.3,-2.13,-1.85,-2.24,-1.62
J0804+2743,1.21924,+08:04:24.97,+27:43:23.2,0.9106,17.45,0.1,12.27,0.06,0.08,-1.96,0.13,0.13,-2.12,-1.79,-2.24,-1.52
J0806+1442,1.21268,+08:06:30.30,+14:42:42.4,1.0943,17.4,0.2,13.51,1.0,1.0,-0.64,1.0,1.0,>-0.80,\nodata,>-0.92,\nodata
J0806+1442,1.21268,+08:06:30.30,+14:42:42.4,0.9238,17.2,0.2,12.57,0.09,0.09,-1.4,0.22,0.22,-1.55,-1.26,-1.66,-1.01
J0810+5025,1.20573,+08:10:02.69,+50:25:38.7,0.65,16.82,0.15,11.75,-1.0,-1.0,-1.81,-1.0,-1.0,\nodata,<-1.67,\nodata,<-1.43


In [11]:
w16_lls = []
for ii,row in enumerate(w16_mtl):
    # L13?
    if ':' not in row['RA']:
        continue
    # init
    lls = LLSSystem(radec=(row['RA']+row['DEC']),zem=row['z_em'],
                    zabs=row['z_abs'],NHI=row['logNHI'],
                    sig_NHI=row['elogNHI'],
                    vlim=[-500,500]*u.km/u.s)
    # Metallicity
    lls.ZH = row['[X/H]']
    lls.sig_ZH = (row['em_[X/H]'], row['ep_[X/H]'])
    # Append
    w16_lls.append(lls)

In [12]:
W16 = LLSSurvey()
W16._abs_sys = w16_lls

In [13]:
W16.ZH

array([-1.99, -1.96, -0.64, -1.4 , -1.81, -0.73, -0.39, -1.08, -1.56,
       -0.17, -1.31, -1.33, -1.12, -0.85, -0.52, -0.92, -0.41, -1.27,
       -2.48, -0.49, -0.44, -1.92, -0.72, -1.79, -1.65, -1.77, -0.23,
       -1.6 , -0.48, -1.66])

## Some fun

In [14]:
from bokeh.io import output_notebook, show, hplot, output_file
from bokeh.plotting import figure
from bokeh.models import Range1d

output_notebook()

Loading BokehJS ...

### NHI

In [15]:
# NHI
p_NHI = figure(plot_width=400, plot_height=400, title='L13/W16 NHI')#,background_fill="#E8DDCB")
# L13
hist, edges = np.histogram(L13.NHI, range=(15.5,18.), density=True, bins=10)
p_NHI.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color='blue')
# W16
hist, edges = np.histogram(W16.NHI, range=(15.5,18.), density=True, bins=10)
p_NHI.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color='red')
#
p_NHI.xaxis.axis_label = 'N_HI'
show(p_NHI)

### Bimodality!

In [16]:
L13W16 = L13 + W16

In [17]:
# NHI
p_ZH = figure(plot_width=600, plot_height=600, title='L13/W16 NHI')#,background_fill="#E8DDCB")
# L13
hist, edges = np.histogram(L13W16.ZH, range=(-2.5,0.5), density=True, bins=20)
p_ZH.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color='blue')
#
p_ZH.xaxis.axis_label = '[Z/H]'
show(p_ZH)

## Write

### L13

In [18]:
L13.write_survey(outfile=pyigm_path+'/data/LLS/Literature/lehner13.tar.gz')

Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/LLS/Literature/lehner13.tar.gz


### W16

In [19]:
W16.write_survey(outfile=pyigm_path+'/data/LLS/Literature/wotta16.tar.gz')

Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/LLS/Literature/wotta16.tar.gz


## Load (test)

In [22]:
l13file = pyigm.__path__[0]+'/data/LLS/Literature/lehner13.tar.gz'
L13_tst = pyiu.load_sys_files(l13file, 'LLS', chk_vel=False, chk_data=False, use_coord=True)

Loading systems from /Users/xavier/local/Python/pyigm/pyigm/data/LLS/Literature/lehner13.tar.gz
Skipping a likely folder: IGM_JSON


In [23]:
L13_tst.NHI

array([ 16.61000061,  16.71999931,  16.45000076,  16.34000015,
        16.89999962,  16.47999954,  16.11000061,  16.20000076,
        16.52000046,  17.        ,  16.39999962,  17.        ,
        16.29999924,  16.45000076,  16.37999916,  16.39999962,
        16.17000008,  16.34000015,  16.64999962,  16.18000031,
        16.21999931,  16.65999985,  16.47999954,  16.97999954,  17.29999924])